In [190]:
import torch
import matplotlib.pyplot as plt
import numpy as np



In [210]:
def mpm(E):
    # nsteps
    nsteps = 5200
    
    # mom tolerance
    tol = 1e-12

    # Domain
    L = 25

    # Material properties
    # E = 100
    rho = 1

    # Computational grid
    nelements = 13 # number of elements
    dx = L / nelements # element length

    # Create equally spaced nodes
    x_n = np.linspace(0, L, nelements+1)
    nnodes = len(x_n)

    # Set-up a 2D array of elements with node ids
    elements = np.zeros((nelements, 2), dtype = int)
    for nid in range(nelements):
        elements[nid,0] = nid
        elements[nid,1] = nid + 1

    # Loading conditions
    v0 = 0.1             # initial velocity
    c  = torch.sqrt(E/rho)  # speed of sound
    b1 = np.pi / (2 * L) # beta1
    w1 = b1 * c          # omega1

    # Create material points at the center of each element
    torcharticles = nelements  # number of particles
    # Id of the particle in the central element
    pmid = 6

    # Material point properties
    x_p      = np.zeros(torcharticles)       # positions
    vol_p    = np.ones(torcharticles) * dx   # volume
    mass_p   = vol_p * rho                 # mass
    stress_p = np.zeros(torcharticles)       # stress
    vel_p    = np.zeros(torcharticles)       # velocity

    # Create particle at the center
    x_p      = 0.5 * (x_n[:-1] + x_n[1:])
    
    # set initial velocities
    vel_p    = v0 * np.sin(b1 * x_p)
    
    # Time steps and duration
    dt_crit = dx / c
    dt = 0.02
    
    # results
    tt = np.zeros(nsteps)
    vt = np.zeros(nsteps)
    xt = np.zeros(nsteps)

    def step(i, carry):
        x_p, mass_p, vel_p, vol_p, stress_p, vt, xt = carry
        # reset nodal values
        mass_n  = np.zeros(nnodes)  # mass
        mom_n   = np.zeros(nnodes)  # momentum
        fint_n  = np.zeros(nnodes)  # internal force

        # iterate through each element
        for eid in range(nelements):
            # get nodal ids
            nid1, nid2 = elements[eid]

            # compute shape functions and derivatives
            N1 = 1 - abs(x_p[eid] - x_n[nid1])/dx
            N2 = 1 - abs(x_p[eid] - x_n[nid2])/dx
            dN1 = -1/dx
            dN2 = 1/dx

            # map particle mass and momentum to nodes
            mass_n[nid1] = mass_n[nid1] + N1 * mass_p[eid]
            mass_n[nid2] = mass_n[nid2] + N2 * mass_p[eid]

            mom_n[nid1] = mom_n[nid1] + N1 * mass_p[eid] * vel_p[eid]
            mom_n[nid2] = mom_n[nid2] + N2 * mass_p[eid] * vel_p[eid]

            # compute nodal internal force
            fint_n[nid1] -= vol_p[eid] * stress_p[eid] * dN1
            fint_n[nid2] -= vol_p[eid] * stress_p[eid] * dN2
        
        # apply boundary conditions
        mom_n[0]  = 0  # Nodal velocity v = 0 in m * v at node 0.
        fint_n[0] = 0 # Nodal force f = m * a, where a = 0 at node 0.

        # update nodal momentum
        mom_n = mom_n + fint_n * dt

        # update particle velocity position and stress
        # iterate through each element
        for eid in range(nelements):
            # get nodal ids
            nid1, nid2 = elements[eid]

            # compute shape functions and derivatives
            N1 = 1 - abs(x_p[eid] - x_n[nid1])/dx
            N2 = 1 - abs(x_p[eid] - x_n[nid2])/dx
            dN1 = -1/dx
            dN2 = 1/dx

            # compute particle velocity
            # if (mass_n[nid1]) > tol:
            vel_p[eid] += dt * N1 * fint_n[nid1] / mass_n[nid1]
            # if (mass_n[nid2]) > tol:
            vel_p[eid] += dt * N2 * fint_n[nid2] / mass_n[nid2]

            # update particle position based on nodal momentum
            x_p[eid] += dt * (N1 * mom_n[nid1]/mass_n[nid1] + N2 * mom_n[nid2]/mass_n[nid2])

            # nodal velocity
            nv1 = mom_n[nid1]/mass_n[nid1]
            nv2 = mom_n[nid2]/mass_n[nid2]

             # rate of strain increment
            grad_v = dN1 * nv1 + dN2 * nv2
            # particle dstrain
            dstrain = grad_v * dt
            # particle volume
            vol_p[eid] *= (1 + dstrain)   
            # update stress using linear elastic model
            stress_p[eid] += E * dstrain
        # results
        vt[i] = vel_p[pmid]
        xt[i] = x_p[pmid]

        return (x_p, mass_p, vel_p, vol_p, stress_p, vt, xt)

    for i in range(0, nsteps):
        x_p, mass_p, vel_p, vol_p, stress_p, vt, xt = step(i, (x_p, mass_p, vel_p, vol_p, stress_p, vt, xt))


    return vt



In [211]:
# Assign target
Etarget = torch.tensor(100)
target = mpm(Etarget)

In [212]:
target

array([ 0.07071068,  0.07069956,  0.07067733, ..., -0.04808849,
       -0.04873531, -0.04937458])

In [213]:
def loss_norm(E):
    vt = mpm(E)
    return torch.norm(vt - target)

In [214]:
E_guess = 95.0

params = torch.nn.ParameterList([torch.tensor(E_guess, requires_grad=True)])


In [215]:
params[0].requires_grad

True

In [216]:
optimizer = torch.optim.Adam(params, lr = 1e-1)


niter = 1000

# training loop
for i in range(niter):
    optimizer.zero_grad()
    loss = loss_norm(params[0])
    loss.backward()
    optimizer.step()



RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
params[0]

Parameter containing:
tensor(95., requires_grad=True)